[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/keras-team/autokeras/blob/master/docs/templates/tutorial/image_classification.ipynb)

In [1]:
import tensorflow as tf
import autokeras as ak
tf.__version__

'2.3.1'

## A Simple Example

### Load MNIST dataset

In [2]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
print('Training image shape:', x_train.shape) # (60000, 28, 28)
print('Training label shape:', y_train.shape) # (60000,)
print('First five training labels:', y_train[:5]) # array([5 0 4 1 9], dtype=uint8)

Training image shape: (60000, 28, 28)
Training label shape: (60000,)
First five training labels: [5 0 4 1 9]


### Run the ImageClassifier

In [3]:
# Initialize the image classifier.
clf = ak.ImageClassifier(max_trials=2) # It tries two different models.

# Feed the image classifier with training data 
# 20% of the data is used as validation data by default for tuning
# the process may run for a bit long time, please try to use GPU
clf.fit(x_train, y_train, epochs=3, verbose=2, ) # each model is trained for three epochs

Trial 2 Complete [00h 10m 52s]
val_loss: 0.33090606331825256

Best val_loss So Far: 0.045517999678850174
Total elapsed time: 00h 11m 23s
INFO:tensorflow:Oracle triggered exit
Epoch 1/3
1875/1875 - 8s - loss: 0.1602 - accuracy: 0.9516
Epoch 2/3
1875/1875 - 8s - loss: 0.0733 - accuracy: 0.9772
Epoch 3/3
1875/1875 - 8s - loss: 0.0586 - accuracy: 0.9816
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: ./image_classifier/best_model/assets


### Get the summarized results during the tuning process (return the best 10 models if existed)

In [4]:
clf.tuner.results_summary()

Results summary
Results in ./image_classifier
Showing 10 best trials
Objective(name='val_loss', direction='min')
Trial summary
Hyperparameters:
image_block_1/block_type: vanilla
image_block_1/normalize: True
image_block_1/augment: False
image_block_1/conv_block_1/kernel_size: 3
image_block_1/conv_block_1/num_blocks: 1
image_block_1/conv_block_1/num_layers: 2
image_block_1/conv_block_1/max_pooling: True
image_block_1/conv_block_1/separable: False
image_block_1/conv_block_1/dropout: 0.25
image_block_1/conv_block_1/filters_0_0: 32
image_block_1/conv_block_1/filters_0_1: 64
classification_head_1/spatial_reduction_1/reduction_type: flatten
classification_head_1/dropout: 0.5
optimizer: adam
learning_rate: 0.001
Score: 0.045517999678850174
Trial summary
Hyperparameters:
image_block_1/block_type: resnet
image_block_1/normalize: True
image_block_1/augment: True
image_block_1/image_augmentation_1/horizontal_flip: True
image_block_1/image_augmentation_1/vertical_flip: True
image_block_1/image_aug

### Retrieve best model

In [5]:
best_model = clf.export_model()
best_model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 28, 28)]          0         
_________________________________________________________________
tf_op_layer_Cast (TensorFlow (None, 28, 28)            0         
_________________________________________________________________
tf_op_layer_ExpandDims (Tens (None, 28, 28, 1)         0         
_________________________________________________________________
normalization (Normalization (None, 28, 28, 1)         3         
_________________________________________________________________
conv2d (Conv2D)              (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 64)       


### Predict with the best model.


In [6]:
predicted_y = clf.predict(x_test)
print(predicted_y)

[['7']
 ['2']
 ['1']
 ...
 ['4']
 ['5']
 ['6']]


In [9]:
type(predicted_y[0][0])

numpy.str_

### Evaluate the best model on the test data.

In [10]:

test_loss, test_acc = clf.evaluate(x_test, y_test, verbose=0)
print('Test accuracy: ', test_acc)


Test accuracy:  0.9873999953269958


### Save and load model

In [17]:
best_model.save("model_autokeras")

INFO:tensorflow:Assets written to: model_autokeras/assets


In [32]:
from tensorflow.keras.models import load_model

loaded_model = load_model("model_autokeras") # , custom_objects=ak.CUSTOM_OBJECTS

predicted_y = loaded_model.predict(tf.expand_dims(x_test, -1))
print(predicted_y)

test_loss, test_acc = clf.evaluate(x_test, y_test, verbose=0)
print('Test accuracy: ', test_acc)

[[2.8646949e-10 6.5158356e-10 2.6511263e-08 ... 9.9999964e-01
  2.8459768e-10 2.5322288e-07]
 [9.5584187e-07 1.8422428e-05 9.9997199e-01 ... 1.7621190e-12
  3.7295683e-07 7.6156220e-11]
 [4.3144240e-08 9.9979836e-01 9.1951561e-06 ... 2.5458250e-05
  5.5137143e-06 1.5064293e-07]
 ...
 [6.0076222e-11 2.4636360e-09 1.6052647e-11 ... 8.7609016e-07
  7.7643826e-06 1.9256097e-06]
 [4.7262091e-07 8.2790198e-11 8.0500290e-10 ... 4.1379331e-09
  8.2674858e-05 1.1726118e-08]
 [8.0499511e-08 2.7859973e-12 7.5505980e-08 ... 1.6911076e-13
  2.5041780e-08 1.9610261e-10]]
Test accuracy:  0.9873999953269958


## Validation Data

By default, AutoKeras use the last 20% of training data as validation data. As shown in the example below, you can use validation_split to specify the percentage.

In [ ]:
clf.fit(x_train,
        y_train,
        # Split the training data and use the last 15% as validation data.
        validation_split=0.15,epochs=3)

You can also use your own validation set instead of splitting it from the training data with validation_data.

In [ ]:
split = 50000
x_val = x_train[split:]
y_val = y_train[split:]
x_train = x_train[:split]
y_train = y_train[:split]
clf.fit(x_train,
        y_train,
        # Use your own validation set.
        validation_data=(x_val, y_val),epochs=3)

## Data Format

The AutoKeras ImageClassifier is quite flexible for the data format.

For the image, it accepts data formats both with and without the channel dimension. The images in the MNIST dataset do not have the channel dimension. Each image is a matrix with shape (28, 28). AutoKeras also accepts images of three dimensions with the channel dimension at last, e.g., (32, 32, 3), (28, 28, 1).

For the classification labels, AutoKeras accepts both plain labels, i.e. strings or integers, and one-hot encoded encoded labels, i.e. vectors of 0s and 1s.

So if you prepare your data in the following way, the ImageClassifier should still work.

In [5]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# Reshape the images to have the channel dimension.
# x_train = x_train.reshape(x_train.shape + (1,))
# x_test = x_test.reshape(x_test.shape + (1,))

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]


# One-hot encode the labels.
import numpy as np
eye = np.eye(10)
y_train = eye[y_train]
y_test = eye[y_test]

print(x_train.shape) # (60000, 28, 28, 1)
print(y_train.shape) # (60000, 10)
print(y_train[:3])
# array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
#        [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
#        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.]])

(60000, 28, 28, 1)
(60000, 10)
[[0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]]


We also support using tf.data.Dataset format for the training data. In this case, the images would have to be 3-dimentional. The labels have to be one-hot encoded for multi-class classification to be wrapped into tensorflow Dataset.

In [7]:
import tensorflow as tf
train_set = tf.data.Dataset.from_tensor_slices(((x_train, ), (y_train, )))
test_set = tf.data.Dataset.from_tensor_slices(((x_test, ), (y_test, )))

clf = ak.ImageClassifier(max_trials=10)
# Feed the tensorflow Dataset to the classifier.
clf.fit(train_set)
# Predict with the best model.
predicted_y = clf.predict(test_set)
# Evaluate the best model with testing data.
print(clf.evaluate(test_set))

TypeError: '>' not supported between instances of 'TensorShape' and 'int'

### Configurate search process

In [5]:
from tensorflow.keras.datasets import mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

clf = ak.ImageClassifier(max_trials=2, 
                         loss='categorical_crossentropy', 
                         metrics=['accuracy'],
                         objective='val_accuracy',
                        )

clf.fit(x_train, y_train, 
        validation_split=0.15,
        epochs=3, verbose=2, )

Train for 1500 steps, validate for 375 steps
Epoch 1/3
1500/1500 - 24s - loss: 0.1769 - accuracy: 0.9466 - val_loss: 0.0683 - val_accuracy: 0.9804
Epoch 2/3
1500/1500 - 18s - loss: 0.0795 - accuracy: 0.9753 - val_loss: 0.0546 - val_accuracy: 0.9849
Epoch 3/3
1500/1500 - 19s - loss: 0.0637 - accuracy: 0.9800 - val_loss: 0.0499 - val_accuracy: 0.9855


Train for 1500 steps, validate for 375 steps
Epoch 1/3
1500/1500 - 120s - loss: 0.2389 - accuracy: 0.9338 - val_loss: 1.1822 - val_accuracy: 0.8832
Epoch 2/3
1500/1500 - 99s - loss: 0.1327 - accuracy: 0.9673 - val_loss: 0.0650 - val_accuracy: 0.9836
Epoch 3/3
1500/1500 - 99s - loss: 0.0719 - accuracy: 0.9802 - val_loss: 0.3224 - val_accuracy: 0.9469


INFO:tensorflow:Oracle triggered exit
Train for 1875 steps, validate for 375 steps
Epoch 1/3
1875/1875 - 24s - loss: 0.1593 - accuracy: 0.9518 - val_loss: 0.0523 - val_accuracy: 0.9833
Epoch 2/3
1875/1875 - 22s - loss: 0.0708 - accuracy: 0.9779 - val_loss: 0.0350 - val_accuracy: 0.9885
Epoch 3/3
1875/1875 - 22s - loss: 0.0605 - accuracy: 0.9814 - val_loss: 0.0218 - val_accuracy: 0.9937


## Reference
[ImageClassifier](/image_classifier),
[AutoModel](/auto_model/#automodel-class),
[ImageBlock](/block/#imageblock-class),
[Normalization](/preprocessor/#normalization-class),
[ImageAugmentation](/preprocessor/#image-augmentation-class),
[ResNetBlock](/block/#resnetblock-class),
[ImageInput](/node/#imageinput-class),
[ClassificationHead](/head/#classificationhead-class).